In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!pip install opencv-python-headless


In [ ]:
import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import numpy as np


In [ ]:
!wget https://github.com/pjreddie/darknet/blob/master/yolov3.weights?raw=true -O yolov3.weights
!wget https://github.com/pjreddie/darknet/blob/master/cfg/yolov3.cfg?raw=true -O yolov3.cfg
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names


In [ ]:

net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")


classes = []
with open("coco.names", "r") as f:
    classes = f.read().strip().split("\n")


In [ ]:
def detect_faces(image):

    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    layer_names = net.getUnconnectedOutLayersNames()
    outs = net.forward(layer_names)


    faces = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]


            if class_id == 0 and confidence > 0.5:
                center_x = int(detection[0] * image.shape[1])
                center_y = int(detection[1] * image.shape[0])
                width = int(detection[2] * image.shape[1])
                height = int(detection[3] * image.shape[0])


                x = int(center_x - width / 2)
                y = int(center_y - height / 2)


                faces.append((x, y, width, height))

    return faces


In [ ]:

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break


    faces = detect_faces(frame)


    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)


    cv2_imshow(frame)


    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()


# !rm yolov3.cfg yolov3.weights coco.names
